In [33]:
import skimage.io
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path
from PIL import Image 
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchvision import models


In [5]:
PATH = Path("../data")

In [6]:
#labels = pd.read_csv(PATH/'train_labels.csv')
test = pd.read_csv(PATH/'test.csv')
labels = pd.read_csv(PATH/'train_labels_clean.csv')
labels.head(3)

,image_id,data_provider,isup_grade,gleason_score
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4


In [7]:
X_train, X_val, y_train, y_val = train_test_split(labels['image_id'].values, 
                                                  labels['isup_grade'].values, test_size=0.2, random_state=42)

In [8]:
def read_file(filename):
    '''return array representing image'''
    return skimage.io.imread(PATH/f'train/{filename}')


In [9]:
def get_mask(filename):
    return skimage.io.imread(PATH/f'masks/{filename}')

In [10]:
class PANDADataset(Dataset):
    def __init__(self, X, y):
        files = []
        for i in range(len(X)):
            files.append(np.concatenate(np.array([read_file(X[i] + '_' + str(j) + '.png') 
                          for j in range(16)])))
        self.x = files
        
        masks = []
        for i in range(len(X)):
            masks.append(np.concatenate(np.array([get_mask(X[i] + '_' + str(j) + '.png') 
                          for j in range(16)])))
        self.y = masks
        
        self.labels = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx], self.labels[idx]
    
train_ds = PANDADataset(X_train[:50], y_train[:50])
valid_ds = PANDADataset(X_val[:50], y_val[:50])

In [11]:
train_dl = DataLoader(train_ds, batch_size=1, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=1)

In [ ]:
conv_block1 =vgg_block_single(in_ch,64)
        self.conv_block2 =vgg_block_single(64,128)

        self.conv_block3 =vgg_block_double(128,256)
        self.conv_block4 =vgg_block_double(256,512)
        self.conv_block5 =vgg_block_double(512,512)

        self.fc_layers

In [57]:
def create_optimizer(model, lr0):
    params = [{'params': model.conv_block1.parameters(), 'lr': lr0/9},
              {'params': model.conv_block2.parameters(), 'lr': lr0/7},
              {'params': model.conv_block3.parameters(), 'lr': lr0/5},
              {'params': model.conv_block4.parameters(), 'lr': lr0/3},
              {'params': model.conv_block5.parameters(), 'lr': lr0/2},
             {'params': model.fc_layers.parameters(), 'lr': lr0}]
    return optim.Adam(params, weight_decay=1e-5)

def update_optimizer(optimizer, group_lrs):
    for i, param_group in enumerate(optimizer.param_groups):
        param_group["lr"] = group_lrs[i]

In [58]:

def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))


def LR_range_finder(model, train_dl, lr_low=1e-4, lr_high=0.01, epochs=2):
    losses = []
    p = PATH/"mode_tmp.pth"
    save_model(model, str(p))
    iterations = epochs * len(train_dl)
    delta = (lr_high - lr_low)/iterations
    lrs = [lr_low + i*delta for i in range(iterations)]
    model.train()
    optimizer = create_optimizer(model, lrs[0])
    ind = 0
    for i in range(epochs):
        for x, y in train_dl:
            lr = lrs[ind]
            update_optimizer(optimizer, [lr/9, lr/7, lr/5,lr/3,lr/2,lr])
            x = x.cuda().float()
            y = y.cuda().float()
            out = model(x)
            loss = F.binary_cross_entropy_with_logits(out, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
            ind +=1
            
    load_model(model, str(p))
    return lrs, losses

In [59]:
def cosine_segment(start_lr, end_lr, iterations):
    i = np.arange(iterations)
    c_i = 1 + np.cos(i*np.pi/iterations)
    return end_lr + (start_lr - end_lr)/2 *c_i

def get_cosine_triangular_lr(max_lr, iterations):
    min_start, min_end = max_lr/25, max_lr/(25*1e4)
    iter1 = int(0.3*iterations)
    iter2 = iterations - iter1
    segs = [cosine_segment(min_start, max_lr, iter1), cosine_segment(max_lr, min_end, iter2)]
    return np.concatenate(segs)

In [60]:
def train_one_epoch(model, optimizer, train_dl, lrs, idx):
    model.train()
    total = 0
    sum_loss = 0
    for x, label, y in train_dl:
        lr = lrs[idx]
        update_optimizer(optimizer, [lr/9, lr/7, lr/5,lr/3,lr/2,lr])
        batch = y.shape[0]
        x = x.cuda().float()
        y = y.cuda().float()
        out = model(x)
        loss = F.cross_entropy(out, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        idx += 1
        total += batch
        sum_loss += batch*(loss.item())
    return sum_loss/total, idx

In [61]:
def train_triangular_policy(model, train_dl, valid_dl, max_lr=0.04, epochs = 5):
    idx = 0
    iterations = epochs*len(train_dl)
    lrs = get_cosine_triangular_lr(max_lr, iterations)
    optimizer = create_optimizer(model, lrs[0])
    prev_val_loss = 10000.0
    for i in range(epochs):
        train_loss, idx = train_one_epoch(model, optimizer, train_dl, lrs, idx)
        val_loss, val_acc = val_metrics(model, valid_dl)
        print("train_loss %.3f val_loss %.3f val_acc %.3f" % (train_loss, val_loss, val_acc))
        if val_loss < prev_val_loss: 
            prev_val_loss = val_loss
            path = "{0}/models/model_resnet34_{1:.0f}.pth".format(PATH, 100*val_acc)
            save_model(model, path)
            print(path)

In [62]:
def val_metrics(model, valid_dl):
    model.eval()
    total = 0
    sum_loss = 0
    correct = 0 
    with torch.no_grad():
        for x, label, y in valid_dl:
            batch = y.shape[0]
            x = x.cuda().float()
            y = y.cuda()
            out = model(x)
            pred = (out > 0.0).long()
            correct += pred.eq(y.data).sum().item()
            y = y.float()
            loss = F.cross_entropy(out, y)
            sum_loss += batch*(loss.item())
            total += batch
        return sum_loss/total, correct/total

In [102]:
x,label,y = next(iter(train_dl))
x = torch.tensor(x).float()

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [103]:
vs = vgg_block_single(2048, 64)
x = vs(x)
x.shape

torch.Size([1, 64, 64, 1])

In [104]:
vs2 = vgg_block_single(64, 128)
x = vs2(x)
x.shape

RuntimeError: Given input size: (128x64x1). Calculated output size: (128x32x0). Output size is too small

In [136]:
def vgg_block_single(in_ch, out_ch, kernel_size=3, padding=1):
    return nn.Sequential(
        nn.Conv2d(in_ch, out_ch, kernel_size=kernel_size, padding=padding),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
        ) 
    
def vgg_block_double(in_ch, out_ch, kernel_size=3, padding=1):
    return nn.Sequential(
        nn.Conv2d(in_ch, out_ch, kernel_size=kernel_size, padding=padding),
        nn.ReLU(),
        nn.Conv2d(out_ch, out_ch, kernel_size=kernel_size, padding=padding),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
        )


class MyVGG11(nn.Module):
    def __init__(self, in_ch, num_classes):
        super().__init__()

        self.conv_block1 = vgg_block_single(in_ch,64)
        self.conv_block2 = vgg_block_single(64,128)

        self.conv_block3 = vgg_block_double(128,256)
        self.conv_block4 = vgg_block_double(256,512)
        self.conv_block5 = vgg_block_double(512,512)

        self.fc_layers = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096), nn.ReLU(inplace=True), nn.Dropout(),
            nn.Linear(4096, 4096), nn.ReLU(inplace=True), nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        
        x = self.conv_block1(x)
        x = self.conv_block2(x)

        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.conv_block5(x)

        x = x.view(x.size(0), -1)

        x = self.fc_layers(x)

        return x

In [137]:
model = MyVGG11(2048, 6).cuda()

In [138]:
train_triangular_policy(model, train_dl, valid_dl, max_lr=0.02, epochs = 5)

RuntimeError: Given input size: (128x64x1). Calculated output size: (128x32x0). Output size is too small